- @Description: Juputer notebook for visualizing depth maps.
- @Author: Zhe Zhang (doublez@stu.pku.edu.cn)
- @Affiliation: Peking University (PKU)
- @LastEditDate: 2023-09-07

In [ ]:
import sys, os
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import re


def read_pfm(filename):
    file = open(filename, 'rb')
    color = None
    width = None
    height = None
    scale = None
    endian = None

    header = file.readline().decode('utf-8').rstrip()
    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode('utf-8'))
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().rstrip())
    if scale < 0:  # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>'  # big-endian

    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)

    data = np.reshape(data, shape)
    data = np.flipud(data)
    file.close()
    return data, scale


def read_depth(filename):
    depth = read_pfm(filename)[0]
    return np.array(depth, dtype=np.float32)


assert False

## DTU

In [ ]:
exp_name = 'dtu/geomvsnet'
depth_name = "00000009.pfm"

scans = os.listdir(os.path.join(exp_name))
scans = list(filter(lambda x: x.startswith("scan"), scans))
scans.sort(key=lambda x: int(x[4:]))
for scan in scans:
    depth_filename = os.path.join(exp_name, scan, "depth_est", depth_name)
    if not os.path.exists(depth_filename): continue
    depth = read_depth(depth_filename)

    confidence_filename = os.path.join(exp_name, scan, "confidence", depth_name)
    confidence = read_depth(confidence_filename)

    print(scan, depth_name)

    plt.figure(figsize=(12, 12))
    plt.subplot(1, 2, 1)
    plt.xticks([]), plt.yticks([]), plt.axis('off')
    plt.imshow(depth, 'viridis',  vmin=500, vmax=830)

    plt.subplot(1, 2, 2)
    plt.xticks([]), plt.yticks([]), plt.axis('off')
    plt.imshow(confidence, 'viridis')
    plt.show()

## TNT

In [ ]:
exp_name = './tnt/blend/geomvsnet/'
depth_name = "00000009.pfm"

with open("../datasets/lists/tnt/intermediate.txt") as f:
    scans_i = [line.rstrip() for line in f.readlines()]

with open("../datasets/lists/tnt/advanced.txt") as f:
    scans_a = [line.rstrip() for line in f.readlines()]

scans = scans_i + scans_a

for scan in scans:

    depth_filename = os.path.join(exp_name, scan, "depth_est", depth_name)
    if not os.path.exists(depth_filename): continue
    depth = read_depth(depth_filename)

    print(scan, depth_name, depth.shape)

    plt.figure(figsize=(12, 12))
    plt.xticks([]), plt.yticks([]), plt.axis('off')
    plt.imshow(depth, 'viridis', vmin=0, vmax=10)

    plt.show()